In [ ]:
%pip install diffusers
%pip install peft
!pip install protobuf==3.20.*
!pip install clean-fid
!pip install open_clip_torch


In [10]:
from pathlib import Path
from diffusers import UNet2DConditionModel, AutoencoderKL, DDPMScheduler, StableDiffusionPipeline, DiffusionPipeline
from transformers import CLIPTextModel, CLIPTokenizer
from huggingface_hub import login
from peft import LoraConfig
import torch
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F
import math
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from peft.utils import get_peft_model_state_dict
from diffusers.utils import convert_state_dict_to_diffusers
from datasets import load_dataset
from functools import partial
from PIL import Image
from kaggle_secrets import UserSecretsClient
from torch.utils.data import Dataset
import pandas as pd
from pydantic import BaseModel
from diffusers.training_utils import compute_snr
import os

2025-11-23 06:50:25.032809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763880625.225711      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763880625.282125      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
df=pd.read_csv('/kaggle/input/fashion-dataset/val_fixed.csv')
df.head()
df = df.iloc[:2000]
captions=df['caption'].tolist()


In [ ]:
import os
import shutil

def copy_first_2000_images(source_folder, destination_folder):
    """
    Sao chép 2000 ảnh đầu tiên từ thư mục nguồn sang thư mục đích
    """
    # Tạo thư mục đích nếu chưa tồn tại
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    # Lấy danh sách tất cả file ảnh trong thư mục nguồn
    all_images = [f for f in os.listdir(source_folder) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
    
    # Sắp xếp danh sách ảnh theo tên
    all_images.sort()
    
    # Lấy 2000 ảnh đầu tiên
    first_2000_images = all_images[:2000]
    
    # Sao chép 2000 ảnh sang thư mục đích
    for image_name in first_2000_images:
        source_path = os.path.join(source_folder, image_name)
        destination_path = os.path.join(destination_folder, image_name)
        shutil.copy2(source_path, destination_path)
    
    print(f"Đã sao chép {len(first_2000_images)} ảnh vào thư mục: {destination_folder}")
    return first_2000_images

# Sử dụng hàm - SAO CHÉP thay vì di chuyển
source_folder = '/kaggle/input/fashion-dataset/val_images'
destination_folder = '/kaggle/working/real_images'

images_list = copy_first_2000_images(source_folder, destination_folder)

# In ra thông tin
print(f"\nTổng số ảnh đã sao chép: {len(images_list)}")
print("\n10 ảnh đầu tiên:")
for i, img in enumerate(images_list[:10]):
    print(f"{i+1}. {img}")

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import os

device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to(device)

save_gen_dir = "generated_images"
os.makedirs(save_gen_dir, exist_ok=True)

generated_paths = []

for i, prompt in enumerate(captions):
    img = pipe(prompt, num_inference_steps=40).images[0]
    save_path = f"{save_gen_dir}/{i:04d}.png"
    img.save(save_path)
    generated_paths.append(save_path)

print("Generated:", len(generated_paths))


In [6]:
from cleanfid import fid

fid_score = fid.compute_fid(
    "/kaggle/input/fidclip-sd1-5-base-generated-images/real_images",
    "/kaggle/input/fidclip-sd1-5-base-generated-images/generated_images",
    mode="clean"
)


print("FID score:", fid_score)


compute FID between two folders


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Found 2000 images in the folder /kaggle/input/fidclip-sd1-5-base-generated-images/real_images


FID real_images : 100%|██████████| 63/63 [01:51<00:00,  1.76s/it]


Found 2000 images in the folder /kaggle/input/fidclip-sd1-5-base-generated-images/generated_images


FID generated_images : 100%|██████████| 63/63 [00:21<00:00,  2.94it/s]


FID score: 62.03515192696102


In [12]:
import os
import torch
import open_clip
from PIL import Image
import numpy as np

# ==============================
# 1. Load CLIP model
# ==============================
device = "cuda" if torch.cuda.is_available() else "cpu"

model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-B-32',
    pretrained='openai'
)
model = model.to(device)

tokenizer = open_clip.get_tokenizer('ViT-B-32')

# ==============================
# 2. Tạo generated_paths từ thư mục ảnh
# ==============================

generated_dir = "/kaggle/input/fidclip-sd1-5-base-generated-images/generated_images"   # <<< sửa theo folder của bạn

generated_paths = sorted([
    os.path.join(generated_dir, f)
    for f in os.listdir(generated_dir)
    if f.lower().endswith((".png", ".jpg", ".jpeg"))
])

print("Số ảnh tìm thấy:", len(generated_paths))

/usr/local/lib/python3.11/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


Số ảnh tìm thấy: 2000


In [13]:


# ==============================
# 3. Load captions (đã có sẵn)
# ==============================
# captions = [...]  # bạn đã có list này từ CSV

assert len(generated_paths) == len(captions), (
    f"Không khớp số lượng! {len(generated_paths)} ảnh nhưng {len(captions)} captions"
)

clip_scores = []

# ==============================
# 4. Loop tính CLIP score hình ↔ caption
# ==============================
for img_path, prompt in zip(generated_paths, captions):

    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
    text = tokenizer([prompt]).to(device)

    with torch.no_grad():
        img_feat = model.encode_image(image)
        txt_feat = model.encode_text(text)

    img_feat = img_feat / img_feat.norm(dim=-1, keepdim=True)
    txt_feat = txt_feat / txt_feat.norm(dim=-1, keepdim=True)

    score = (img_feat @ txt_feat.T).item()
    clip_scores.append(score)

# ==============================
# 5. Output
# ==============================
avg_score = float(np.mean(clip_scores))
print("Số lượng samples:", len(clip_scores))
print("Average CLIP score:", avg_score)


Số lượng samples: 2000
Average CLIP score: 0.3164151344001293
